<a href="https://colab.research.google.com/github/lschuler-dotcom/capstone_project/blob/main/notebooks/capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Split the raw data file into 4 for SQL cleaning and synthetic analysis**

In [3]:
from google.colab import files
uploaded = files.upload()

Saving v1_social_media_vs_productivity.csv to v1_social_media_vs_productivity.csv


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('v1_social_media_vs_productivity.csv')
df.head()

,user_id,age,gender,job_type,daily_social_media_time,social_platform_preference,number_of_notifications,work_hours_per_day,perceived_productivity_score,actual_productivity_score,stress_level,sleep_hours,screen_time_before_sleep,breaks_during_work,uses_focus_apps,has_digital_wellbeing_enabled,coffee_consumption_per_day,days_feeling_burnout_per_month,weekly_offline_hours,job_satisfaction_score
0,1,56,Male,Unemployed,4.180940,Facebook,61,6.753558,8.040464,7.291555,4.0,5.116546,0.419102,8,False,False,4,11,21.927072,6.336688
1,2,46,Male,Health,3.249603,Twitter,59,9.169296,5.063368,5.165093,7.0,5.103897,0.671519,7,True,True,2,25,0.000000,3.412427
2,3,32,Male,Finance,NaN,Twitter,57,7.910952,3.861762,3.474053,4.0,8.583222,0.624378,0,True,False,3,17,10.322044,2.474944
3,4,60,Female,Unemployed,NaN,Facebook,59,6.355027,2.916331,1.774869,6.0,6.052984,1.204540,1,False,False,0,4,23.876616,1.733670
4,5,25,Male,IT,NaN,Telegram,66,6.214096,8.868753,NaN,7.0,5.405706,1.876254,1,False,True,1,30,10.653519,9.693060


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   user_id                         30000 non-null  int64  
 1   age                             30000 non-null  int64  
 2   gender                          30000 non-null  object 
 3   job_type                        30000 non-null  object 
 4   daily_social_media_time         27235 non-null  float64
 5   social_platform_preference      30000 non-null  object 
 6   number_of_notifications         30000 non-null  int64  
 7   work_hours_per_day              30000 non-null  float64
 8   perceived_productivity_score    28386 non-null  float64
 9   actual_productivity_score       27635 non-null  float64
 10  stress_level                    28096 non-null  float64
 11  sleep_hours                     27402 non-null  float64
 12  screen_time_before_sleep        

In [20]:
df.describe()

,user_id,age,daily_social_media_time,number_of_notifications,work_hours_per_day,perceived_productivity_score,actual_productivity_score,stress_level,sleep_hours,screen_time_before_sleep,breaks_during_work,coffee_consumption_per_day,days_feeling_burnout_per_month,weekly_offline_hours,job_satisfaction_score
count,30000.000000,30000.000000,27235.000000,30000.000000,30000.000000,28386.000000,27635.000000,28096.000000,27402.000000,27789.000000,30000.000000,30000.000000,30000.000000,30000.000000,27270.000000
mean,15000.500000,41.486867,3.113418,59.958767,6.990792,5.510488,4.951805,5.514059,6.500247,1.025568,4.992200,1.999300,15.557067,10.360655,4.964901
std,8660.398374,13.835221,2.074813,7.723772,1.997736,2.023470,1.883378,2.866344,1.464004,0.653355,3.173737,1.410047,9.252956,7.280415,2.121194
min,1.000000,18.000000,0.000000,30.000000,0.000000,2.000252,0.296812,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,30.000000,1.639566,55.000000,5.643771,3.757861,3.373284,3.000000,5.493536,0.528490,2.000000,1.000000,8.000000,4.541872,3.363580
50%,15000.500000,41.000000,3.025913,60.000000,6.990641,5.525005,4.951742,6.000000,6.498340,1.006159,5.000000,2.000000,16.000000,10.013677,4.951049
75%,22500.250000,53.000000,4.368917,65.000000,8.354725,7.265776,6.526342,8.000000,7.504143,1.477221,8.000000,3.000000,24.000000,15.300809,6.581323
max,30000.000000,65.000000,17.973256,90.000000,12.000000,8.999376,9.846258,10.000000,10.000000,3.000000,10.000000,10.000000,31.000000,40.964769,10.000000


In [8]:
df.columns

Index(['user_id', 'age', 'gender', 'job_type', 'daily_social_media_time',
       'social_platform_preference', 'number_of_notifications',
       'work_hours_per_day', 'perceived_productivity_score',
       'actual_productivity_score', 'stress_level', 'sleep_hours',
       'screen_time_before_sleep', 'breaks_during_work', 'uses_focus_apps',
       'has_digital_wellbeing_enabled', 'coffee_consumption_per_day',
       'days_feeling_burnout_per_month', 'weekly_offline_hours',
       'job_satisfaction_score'],
      dtype='object')

In [13]:
user_demographics = df[[
    'user_id','age','gender','job_type',
    'coffee_consumption_per_day','sleep_hours',
    'work_hours_per_day','breaks_during_work',
    'weekly_offline_hours'
]]

In [ ]:
digital_habits = df[[
    'user_id',
    'daily_social_media_time',
    'social_platform_preference',
    'number_of_notifications',
    'screen_time_before_sleep',
    'uses_focus_apps'
]]

In [ ]:
well_being = df[[
    'user_id','stress_level',
    'has_digital_wellbeing_enabled','days_feeling_burnout_per_month'
]]

In [14]:
productivity_scores = df[[
  'user_id',
  'perceived_productivity_score','actual_productivity_score'
]]

**Export new CSV's**

In [ ]:
user_demographics.to_csv('user_demographics')